In [ ]:
#!/usr/bin/env python3
"""
planner_controller.py

- Subscribe to triangulation PUB (default tcp://localhost:5566)
- Compute car center using tag poses (pos + yaw). If both tags visible -> midpoint.
- Compute projectile intercept or free-fall fallback to z=0.20 m.
- Simple speed estimation (half-throttle), then proportional throttle control.
- Send throttle commands to ESP32 via serial (classic BT SPP, e.g. /dev/rfcomm0).
"""

import zmq, time, math, argparse, threading, collections, serial, sys, numpy as np

# Config
TRIANG_ADDR = "tcp://localhost:5566"
SERIAL_PORT = "/dev/rfcomm0"   # set to your paired RFCOMM device
BAUD = 115200
TARGET_Z = 0.20
GRAV = 9.81
VEL_EST_WINDOW = 0.12
MIN_SAMPLES = 3
SPEED_EST_DURATION = 0.6
HALF_THROTTLE = 50       # percent (0..100)
P_GAIN = 0.8
CAR_OFFSET_TAG4 = (0.0, 0.96)   # offset of tag4 in robot frame (x forward, y right) — adjust if needed
CAR_OFFSET_TAG5 = (0.0, -0.96)
MAX_THROTTLE = 100
MIN_THROTTLE = -100

# helpers
def now(): return time.time()

def estimate_velocity(points):
    # points: list of (ts, x, y, z) sorted by ts
    if len(points) < 2: return None
    ts = np.array([p[0] for p in points], dtype=float)
    xs = np.array([p[1] for p in points], dtype=float)
    ys = np.array([p[2] for p in points], dtype=float)
    zs = np.array([0.0 if (p[3] is None or math.isnan(p[3])) else p[3] for p in points], dtype=float)
    t0 = ts[0]
    dt = ts - t0
    def slope(arr):
        A = np.vstack([dt, np.ones_like(dt)]).T
        m, _ = np.linalg.lstsq(A, arr, rcond=None)[0]
        return float(m)
    return slope(xs), slope(ys), slope(zs)

def solve_time_to_z(z0, vz0, target_z=TARGET_Z, g=GRAV):
    a = -0.5*g
    b = vz0
    c = z0 - target_z
    disc = b*b - 4*a*c
    if disc < 0: return None
    r1 = (-b + math.sqrt(disc)) / (2*a)
    r2 = (-b - math.sqrt(disc)) / (2*a)
    ts = [t for t in (r1, r2) if t > 0]
    if not ts: return None
    return min(ts)

class CarSerial:
    def __init__(self, port=SERIAL_PORT, baud=BAUD, timeout=0.5):
        self.port = port
        self.baud = baud
        try:
            self.ser = serial.Serial(port, baud, timeout=timeout)
            time.sleep(0.2)
            print(f"[CarSerial] Opened {port} @ {baud}")
        except Exception as e:
            print("[CarSerial] Failed to open serial:", e)
            self.ser = None
    def send_throttle(self, val):
        # val: -100..100 int
        v = int(max(MIN_THROTTLE, min(MAX_THROTTLE, round(val))))
        cmd = f"T{v}\n".encode('ascii')
        if self.ser:
            self.ser.write(cmd)
        print(f"[CarSerial] -> {cmd!r}")
    def request_odometry(self):
        # returns dx, dy in meters or (0,0)
        if not self.ser:
            return 0.0, 0.0
        try:
            self.ser.write(b"O\n")
            line = self.ser.readline().decode('ascii').strip()
            if line.startswith("ODOM:"):
                parts = line.split(":")[1].split(",")
                dx = float(parts[0]); dy = float(parts[1])
                return dx, dy
        except Exception as e:
            # timeout or parse error
            return 0.0, 0.0
        return 0.0, 0.0

class Planner:
    def __init__(self, zmq_addr=TRIANG_ADDR, serial_port=SERIAL_PORT):
        self.ctx = zmq.Context()
        self.sub = self.ctx.socket(zmq.SUB)
        self.sub.connect(zmq_addr)
        self.sub.setsockopt_string(zmq.SUBSCRIBE, "")
        self.car = CarSerial(serial_port)
        self.ball_buf = collections.deque()
        self.buf_lock = threading.Lock()
        self.last_car_center = (0.0, 0.0)
        self.estimated_speed = None
        self.speed_done = False

    def start(self):
        threading.Thread(target=self.recv_loop, daemon=True).start()
        self.control_loop()

    def recv_loop(self):
        while True:
            try:
                msg = self.sub.recv_json(flags=0)
            except Exception:
                time.sleep(0.005); continue
            ts = msg.get("ts", time.time())
            b = msg.get("ball", {})
            t4 = msg.get("tag4", {})
            t5 = msg.get("tag5", {})
            # push ball xy/z to buffer if xy available
            bx = b.get("x"); by = b.get("y"); bz = b.get("z")
            if bx is not None and by is not None:
                with self.buf_lock:
                    self.ball_buf.append((ts, float(bx), float(by), float(bz) if bz is not None else float('nan')))
                    while len(self.ball_buf) > 60: self.ball_buf.popleft()
            # compute car center from tags
            car_center = None
            if t4.get("valid") and t5.get("valid"):
                p4 = t4["pos"]; p5 = t5["pos"]
                car_center = ((p4[0]+p5[0])/2.0, (p4[1]+p5[1])/2.0)
            elif t4.get("valid"):
                p4 = t4["pos"]; yaw = t4.get("yaw", None)
                if yaw is not None:
                    dx = CAR_OFFSET_TAG4[0]*math.cos(yaw) - CAR_OFFSET_TAG4[1]*math.sin(yaw)
                    dy = CAR_OFFSET_TAG4[0]*math.sin(yaw) + CAR_OFFSET_TAG4[1]*math.cos(yaw)
                    car_center = (p4[0] - dx, p4[1] - dy)
            elif t5.get("valid"):
                p5 = t5["pos"]; yaw = t5.get("yaw", None)
                if yaw is not None:
                    dx = CAR_OFFSET_TAG5[0]*math.cos(yaw) - CAR_OFFSET_TAG5[1]*math.sin(yaw)
                    dy = CAR_OFFSET_TAG5[0]*math.sin(yaw) + CAR_OFFSET_TAG5[1]*math.cos(yaw)
                    car_center = (p5[0] - dx, p5[1] - dy)
            if car_center is not None:
                self.last_car_center = (float(car_center[0]), float(car_center[1]))

    def control_loop(self):
        rate = 10.0
        period = 1.0 / rate
        while True:
            t0 = now()
            with self.buf_lock:
                buf = list(self.ball_buf)
            if not buf:
                time.sleep(period); continue
            recent = [p for p in buf if (buf[-1][0] - p[0]) <= VEL_EST_WINDOW]
            vel_est = estimate_velocity(recent) if len(recent) >= 2 else None
            vx = vy = vz = 0.0
            if vel_est is not None:
                vx, vy, vz = vel_est
            latest = buf[-1]
            ts, bx, by, bz_raw = latest
            bz = None if math.isnan(bz_raw) else bz_raw

            # predictive intercept attempt
            target_xy = None
            intercept_t = None
            if bz is not None and vel_est is not None and len(buf) >= MIN_SAMPLES:
                t_hit = solve_time_to_z(bz, vz)
                if t_hit is not None and t_hit > 0.02:
                    target_xy = (bx + vx*t_hit, by + vy*t_hit)
                    intercept_t = t_hit
            # fallback free-fall
            if target_xy is None:
                if bz is None:
                    # can't compute height -> aim to current xy (no z)
                    target_xy = (bx, by)
                    intercept_t = 0.0
                else:
                    t_hit = solve_time_to_z(bz, 0.0)
                    if t_hit is None: t_hit = 0.0
                    target_xy = (bx, by)
                    intercept_t = t_hit

            # clamp into arena square [0,0]..[0.9,0.9]
            tx = max(0.0, min(0.9, target_xy[0]))
            ty = max(0.0, min(0.9, target_xy[1]))
            target_xy = (tx, ty)

            # compute control error
            car_x, car_y = self.last_car_center
            err_x = target_xy[0] - car_x
            err_y = target_xy[1] - car_y
            dist = math.hypot(err_x, err_y)

            # estimate speed (once)
            if not self.speed_done:
                # do half-throttle estimate
                print("[Planner] Estimating speed with half-throttle...")
                self.car.send_throttle(HALF_THROTTLE)
                time.sleep(SPEED_EST_DURATION)
                dx, dy = self.car.request_odometry()
                moved = math.hypot(dx, dy)
                if moved > 1e-4:
                    self.estimated_speed = moved / SPEED_EST_DURATION
                else:
                    self.estimated_speed = 0.05
                self.speed_done = True
                print(f"[Planner] estimated speed {self.estimated_speed:.3f} m/s")

            # proportional control: compute desired speed proportional to dist
            desired_speed = P_GAIN * dist
            # map desired_speed to throttle using estimated_speed (rough linear)
            if self.estimated_speed and self.estimated_speed > 1e-6:
                throttle = (desired_speed / self.estimated_speed) * 100.0
            else:
                throttle = min(100.0, desired_speed*50.0)

            # direction: no steering implemented here; assume car oriented roughly towards target
            # We send throttle sign positive only (assumes car control handles direction separately).
            # For differential drive you must replace this with steering commands.
            throttle_cmd = max(MIN_THROTTLE, min(MAX_THROTTLE, throttle))

            if dist < 0.02:
                # stop
                self.car.send_throttle(0)
            else:
                self.car.send_throttle(throttle_cmd)

            print(f"[Planner] t={now():.3f} target={target_xy} car={self.last_car_center} err={dist:.3f} t_hit={intercept_t:.3f} thr={throttle_cmd:.1f}")
            dt = now() - t0
            if dt < period:
                time.sleep(period - dt)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--triang", default=TRIANG_ADDR, help="ZMQ triangulation PUB address")
    parser.add_argument("--serial", default=SERIAL_PORT, help="Serial device for BT RFCOMM (e.g. /dev/rfcomm0)")
    args = parser.parse_args()
    TRIANG_ADDR = args.triang
    SERIAL_PORT = args.serial
    p = Planner(TRIANG_ADDR, SERIAL_PORT)
    p.start()


In [1]:
import socket

SERVER_IP = "0.0.0.0"    # listen on all interfaces
SERVER_PORT = 8000       # must match ESP32 client

print("Starting TCP server...")
srv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
srv.bind((SERVER_IP, SERVER_PORT))
srv.listen(1)

print("Waiting for ESP32 to connect...")

conn, addr = srv.accept()
print("ESP32 CONNECTED from", addr)

# send a test command immediately:
conn.sendall(b"T50\n")    # throttle 50 test
# simple echo loop
while True:
    data = conn.recv(1024)
    if not data:
        print("ESP32 disconnected.")
        break
    print("Received from ESP32:", data.decode().strip())
    # Reply something
    conn.sendall(b"ACK_FROM_LAPTOP\n")


Starting TCP server...
Waiting for ESP32 to connect...
ESP32 CONNECTED from ('10.42.0.143', 53247)
Received from ESP32: ACK:T50
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: ERR
Received from ESP32: E

KeyboardInterrupt: 

In [6]:
# Notebook cell 2 — minimal BT send/receive test (blocking read)
import serial, time

# === CONFIG ===
PORT = "/dev/rfcomm0"   # replace with COM# on Windows like "COM5"
BAUD = 115200
TIMEOUT = 1.0           # seconds to wait for a reply

# === open serial ===
try:
    ser = serial.Serial(PORT, BAUD, timeout=TIMEOUT)
    print("Opened serial:", ser.name)
except Exception as e:
    print("Failed to open serial:", e)
    raise

def send_and_recv(cmd, wait=0.05):
    """Send ASCII command (without newline) and read one response line."""
    ser.write((cmd + "\n").encode('ascii'))
    # short sleep to let device respond
    time.sleep(wait)
    # read all available lines
    lines = []
    # try to read multiple lines quickly
    start = time.time()
    while True:
        line = ser.readline().decode('ascii', errors='ignore').strip()
        if line:
            lines.append(line)
        if (time.time() - start) > TIMEOUT:
            break
    return lines

# Test: send throttle 50 and request odometry
print("-> Sending throttle T50")
resp = send_and_recv("T50")
print("Replies:", resp)

print("-> Requesting odometry O")
resp = send_and_recv("O")
print("Replies:", resp)

# close when done
ser.close()


Opened serial: /dev/rfcomm0
-> Sending throttle T50
Replies: []
-> Requesting odometry O


SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)